In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2
import re
import collections

kaggle_username = "rahaf8"
kaggle_key = "f59b8cb26f2973bc6fb4c52b1516ac19"
os.environ["KAGGLE_USERNAME"] = kaggle_username
os.environ["KAGGLE_KEY"] = kaggle_key
import kaggle

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_kaggle_dataset(dataset_name):
    """Get dataset from Kaggle.

    Args:
        dataset_name: the dataset name.
    """

    # Download the dataset using the Kaggle API
    kaggle.api.dataset_download_files(dataset_name, path=".", unzip=True)

In [3]:
def video_dataframe(data_dir):
  """Get video dataframe.

    Args:
      files_path: A path from which the files can be stored.

    Returns:
      Video dataframe containing the labels , videos name , and videos path.
  """
  vidDf = pd.DataFrame(columns=['Lable','VidName','VidPath'])

  for dirname, _, filenames in os.walk(data_dir):
      for name in filenames:
            vidDf =  vidDf.append({'Lable': re.match(r'^[^\d_]+', name).group(),
                                   'VidName': name,
                                   'VidPath': os.path.join(dirname, name)},
                                    ignore_index=True)
  return vidDf

In [6]:
dataset_name = "shashiprakash204/ucfcrimeminidataset"
dataset_dir = "/content/dataset/"

get_kaggle_dataset(dataset_name)

In [8]:
video_df = video_dataframe(dataset_dir)
classes = video_df['Lable'].unique().tolist()
print('Number of classes', len(classes))
print('Num videos for each class: : ')
print(video_df['Lable'].value_counts())

Number of classes 6
Num videos for each class: : 
Fighting    7
Arrest      7
Burglary    7
Abuse       7
Assault     7
Normal      7
Name: Lable, dtype: int64
